Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# PyTorch Pretrained BERT on AzureML with GLUE Dataset
This notebook contains an end-to-end walkthrough of using Azure Machine Learning Service to run [PyTorch reimplementation](https://github.com/huggingface/pytorch-pretrained-BERT) of [Google's TensorFlow repository for the BERT model](https://github.com/google-research/bert) developed by Hugging Face.

You will find the following contents:
- Download GLUE dataset on the remote compute and store them in Azure storage
- Speep-up fine-tuning BERT for GLUE dataset on AzureML GPU clusters
- Further fine-tune BERT wtih AzureML hyperparameter optimizer 

## Prerequisites
- Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)

- Install the Python SDK:  make sure to install notebook, and contrib
```
conda create -n azureml -y Python=3.6
source activate azureml
pip install --upgrade azureml-sdk[notebooks,contrib] 
conda install ipywidgets
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable azureml.widgets --user --py
```

You will need to restart jupyter after this
Detailed instructions are here: https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python 

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize workspace

To create or access an Azure ML Workspace, you will need to import the AML library and the following information:
* A name for your workspace
* Your subscription id
* The resource group name

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step or create a new one. 

In [ ]:
from azureml.core.workspace import Workspace

workspace_name = ''
subscription_id = ''
resource_group_name = ''
location = ''

ws = Workspace._get_or_create(workspace_name,
                             subscription_id=subscription_id,
                             resource_group=resource_group_name,
                             location=location)

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'BERT-GLUE'
experiment = Experiment(ws, name=experiment_name)

## Download GLUE dataset on the remote compute

Before we start to fine-tune the pretained BERT model, we need to download the [GLUE data](https://gluebenchmark.com/tasks) by running the [script](https://gist.github.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e) and unpack it to an Azure Blob container.

### Define AzureML datastore to collect training dataset

To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets.

Each workspace is associated with a default Azure Blob datastore named `'workspaceblobstore'`. In this work, we use this default datastore to collect the GLUE training dataset .

In [ ]:
from azureml.core import Datastore
ds = Datastore(ws, 'workspaceblobstore')

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [ ]:
import os

project_folder = './pytorch-pretrained-BERT'

Make a local clone of the original [PyTorch reimplementation](https://github.com/huggingface/pytorch-pretrained-BERT) repository

In [ ]:
!git clone -b v0.4.0 https://github.com/huggingface/pytorch-pretrained-BERT.git

We need to run the [script](https://gist.github.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e) to download the [GLUE data](https://gluebenchmark.com/tasks) in the mounted Azure Blob container. In our example, we only download `MRPC` dataset

In [ ]:
import urllib, os
urllib.request.urlretrieve( 'https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/becd574dd938f045ea5bd3cb77d1d506541b5345/download_glue_data.py', filename='./download_glue_data.py')
import download_glue_data
download_glue_data.main(["--tasks", "MRPC", "--data_dir","./glue"])

Please note that, if you receive `UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1183: character maps to <undefined>`. Please modify all `'with open(...)'` operations in the downloaded `download_glue_data.py` to `'open(.., encoding="utf8")'`

The following code will upload the training data to the path ./glue on the default datastore.

In [ ]:
ds.upload(src_dir='./glue', target_path='./glue')

## Fine-tuning BERT with Distributed Training
As our `GLUE` dataset are ready in Azure storage, we can start the fine-tune the model by exploting the power of distributed training. 

### Create a GPU remote compute target

We need to create a GPU [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to perform the fine-tuning. In this example, we create an AmlCompute cluster as our training compute resource. Please find the information of Azure VM size in below table.


|    VM Size    	| CPU 	|   GPU   	| Storage (SSD) 	| GPU memory 	| InfiniBand  	|
|:-------------:	|:---:	|:-------:	|:-------------:	|:----------:	|:----------:	|
|  Standard_NC6 	|  6  	| 1 x K80 	|    340 GiB    	|    8 GiB   	|      No   	|
| Standard_NC12 	|  12 	| 2 x K80 	|    680 GiB    	|   16 GiB   	|      No   	|
| Standard_NC24 	|  24 	| 4 x K80 	|    1440 GiB   	|   32 GiB   	|      No   	|
| Standard_NC24r 	|  24 	| 4 x K80 	|    1440 GiB   	|   32 GiB   	|      Yes   	|
| Standard_NC6s_v3 	|  6  	| 1 x V100 	|    736 GiB    	|   16 GiB   	|      No   	|
| Standard_NC12s_v3 |  12 	| 2 x V100 	|    1474 GiB   	|   32 GiB   	|      No   	|
| Standard_NC24s_v3 |  24 	| 4 x V100 	|    2948 GiB   	|   64 GiB   	|      No   	|
| Standard_NC24rs_v3|  24 	| 4 x V100 	|    2948 GiB   	|   64 GiB   	|      Yes   	|


***Note that*** you need to request NCv3-serie quota if you would like to use NVIDIA Tesla V100 

This code creates a cluster for you if it does not already exist in your workspace.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
gpu_cluster_name = "nc24Cluster"

try:
    gpu_compute_target = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24', max_nodes=4)

    # create the cluster
    gpu_compute_target = ComputeTarget.create(ws, gpu_cluster_name, compute_config)
    gpu_compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(gpu_compute_target.status.serialize())

### Create a PyTorch estimator for fine-tuning
Let us create a new PyTorch estimator to run the fine-tuning script `run_classifier.py`, that is already provided at [the original repository](https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_classifier.py). Please refer [here](https://github.com/huggingface/pytorch-pretrained-BERT#fine-tuning-with-bert-running-the-examples) for more detail about the script. 

The original `run_classifier.py` script uses PyTorch distributed launch untility to launch multiple processes across nodes and GPUs. We prepared a modified version [run_classifier_azureml.py](./run_classifier_azureml.py) so that we can launch it based on AzureML build-in MPI backend.

To use AML's tracking and metrics capabilities, we need to add a small amount of AzureML code inside the training script.

In `run_classifier_azureml.py`, we will log some metrics to our AML run. To do so, we will access the AML run object within the script:
```Python
from azureml.core.run import Run
run = Run.get_context()
```
Further within `run_classifier_azureml.py`, we log learning rate, training loss and evaluation accuracy the model achieves as:
```Python
run.log('lr', np.float(args.learning_rate))
...

for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")): 
    ...
    run.log('train_loss', np.float(loss))

...

result = {'eval_loss': eval_loss,
          'eval_accuracy': eval_accuracy}
for key in sorted(result.keys()):
    run.log(key, str(result[key]))
```
These run metrics will become particularly important when we begin hyperparameter tuning our model in the "Tune model hyperparameters" section.

Let's first copy the training script `run_classifier_azureml.py` into our project directory.

In [ ]:
import shutil
shutil.copy('run_classifier_azureml.py', project_folder)
shutil.copy('azureml_bert_util.py', project_folder)

Then, AzureML PyTorch estimator can be defined as below. We use `azuremlsamples/bert:torch-1.0.0-apex-cuda9` as the base docker image with [dockerfile](./dockerfile)

In [ ]:
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder,
                    compute_target=gpu_compute_target,
                    script_params = {
                          '--bert_model':'bert-base-cased',
                          '--task_name': 'MRPC',
                          '--data_dir': ds.path('glue/MRPC/').as_mount(),
                          '--do_train' : '',
                          '--do_eval': '',
                          '--max_seq_length': 128,
                          '--train_batch_size': 32,
                          '--learning_rate': 2e-5,
                          '--num_train_epochs': 3.0,
                          '--output_dir': './outputs',
                          '--seed':16
                    },
                    custom_docker_base_image='azuremlsamples/bert:torch-1.0.0-apex-cuda9',
                    entry_script='run_classifier_azureml.py',
                    node_count=1,
                    process_count_per_node=4,
                    distributed_backend='mpi',
                    use_gpu=True)

estimator._estimator_config.environment.python.user_managed_dependencies=True

### Submit and Monitor your run

In [ ]:
run = experiment.submit(estimator)
from azureml.widgets import RunDetails
RunDetails(run).show()

To achieve an average of **85% evaluation accuracy** with `MRPC dataset`, it requires **3** epochs when fine-tune with `BERT base` model. Below please find the elapsed time per epoch using deferent Azure GPU VMs with above hyperparameters

|  GPU counts 	|    1 GPU    	|         2 GPU 	| 4 GPU      	|
|------------:	|:-----------:	|--------------:	|------------	|
|   NC-series 	| 191 s/epoch 	| 105 s/epoch   	| 60 s/epoch 	|
| NCv3-series 	|  36 s/epoch 	|    22 s/epoch 	| 13 s/epoch 	|
| NCv3 with fp16|  32 s/epoch 	|    18 s/epoch 	| 12 s/epoch 	|

## Fine-Tuning BERT with Hyperparameter Tuning

We would also like to optimize our hyperparameter, `learning rate`, using Azure Machine Learning's hyperparameter tuning capabilities.

### Start a hyperparameter sweep
First, we will define the hyperparameter space to sweep over. In this example we will use random sampling to try different configuration sets of hyperparameter to minimize our primary metric, the evaluation accuracy (`eval_accuracy`).

In [ ]:
from azureml.train.hyperdrive import *
import math

param_sampling = RandomParameterSampling( {
        'learning_rate': loguniform(math.log(1e-4), math.log(1e-6)),
    }
)

hyperdrive_run_config = HyperDriveRunConfig(estimator=estimator,
                                            hyperparameter_sampling=param_sampling, 
                                            primary_metric_name='eval_accuracy',
                                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs=16,
                                            max_concurrent_runs=4)

Finally, lauch the hyperparameter tuning job.

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

### Monitor HyperDrive runs
We can monitor the progress of the runs with the following Jupyter widget. 

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

### Find and register the best model
Once all the runs complete, we can find the run that produced the model with the highest evaluation accuracy.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best Run is:\n  accuracy: {0:.5f} \n  Learning rate: {1:.8f}'.format(
        best_run_metrics['eval_accuracy'][-1],
        best_run_metrics['lr']
     ))

You can compare the resulting optimal `learning_rate` with the value suggested by the [original implementation hyper-parameters](https://github.com/google-research/bert#sentence-and-sentence-pair-classification-tasks): 2e-5